In [ ]:
import pandas as pd
import re

In [ ]:
df_raw = pd.read_csv("C:/Users/Janak ARORA/Desktop/Ironhack_files/Week_7/Day1/machine_learning_project/data/raw/Amazon.csv")
df_raw.head()

In [ ]:
df_raw.dtypes

In [ ]:
df_raw.shape

In [ ]:
df_raw.columns

In [ ]:
df_raw['ship-postal-code'].unique()

In [ ]:
df_raw = df_raw.drop(columns=["index", "Unnamed: 22", "Sales Channel ", "currency", "fulfilled-by", "ship-country", "promotion-ids"])
df_raw.head()

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_cleaned = df_raw.dropna()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
df_cleaned["Date"] = pd.to_datetime(df_cleaned["Date"], errors="coerce")

In [ ]:
df_cleaned["ship-postal-code"] = (
    pd.to_numeric(df_cleaned["ship-postal-code"], errors="coerce")
    .fillna(0)
    .astype(int)
)

In [ ]:
df_cleaned["Order ID"] = df_cleaned["Order ID"].str.replace("-", "", regex=False)

In [ ]:
df_cleaned.dtypes

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.to_csv("cleaned_amazon_data.csv", index=False)

### Clean City Column

In [ ]:
df_cleaned["ship-city"].unique()

In [ ]:
unique_cities = sorted(df_cleaned["ship-city"].dropna().unique())
len(unique_cities), unique_cities[:50]

In [ ]:
# Step 1: normalize city text
def normalize_city(city):
    city = str(city).lower().strip()
    city = re.sub(r"[^a-z\s]", " ", city)   # remove symbols/numbers
    city = re.sub(r"\s+", " ", city)        # collapse spaces
    return city

def camel_with_space(city):
    return " ".join(word.capitalize() for word in city.split())
    
# Step 2: explicit duplicate / variant city merges
city_mapping = {
    # Bangalore
    "bengaluru": "bangalore",
    "banglore": "bangalore",
    "blr": "bangalore",

    # Delhi NCR
    "new delhi": "new delhi",
    "delhi ncr": "delhi",
    "ncr": "delhi",

    # Mumbai
    "bombay": "mumbai",
    "andar": "mumbai",
    "andhari": "mumbai",
    "andheri": "mumbai",
    "andheri e": "mumbai",
    "andheri east": "mumbai",
    "andheri east mumbai": "mumbai",
    "andheri west mumbai": "mumbai",
    
    # Chennai
    "madras": "chennai",

    # Kolkata
    "calcutta": "kolkata",

    # Hyderabad
    "secunderabad": "hyderabad",

    # Kerala
    "cochin": "kochi",
    "trivandrum": "thiruvananthapuram",

    # Gurgaon / Gurugram
    "gurugram": "gurgaon",

    # Noida
    "greater noida": "noida",
    "noida greater noida": "noida"
}

df_cleaned["ship-city"] = df_cleaned["ship-city"].apply(normalize_city)
df_cleaned["ship-city"] = df_cleaned["ship-city"].replace(city_mapping)
df_cleaned["ship-city"] = df_cleaned["ship-city"].apply(camel_with_space)

In [ ]:
cleaned_unique_cities = sorted(df_cleaned["ship-city"].unique())

print("Unique cities BEFORE:", len(unique_cities))
print("Unique cities AFTER:", len(cleaned_unique_cities))
cleaned_unique_cities

In [ ]:
df_cleaned["ship-city"].value_counts()

### Clean State Column

In [ ]:
df_cleaned["ship-state"].unique()

In [ ]:
# Capture original unique states
original_states = sorted(df_cleaned["ship-state"].dropna().unique())

# Normalize state names
df_cleaned["ship-state"] = df_cleaned["ship-state"].astype(str).str.strip().str.lower()

# Known manual merges for common duplicates / variants
state_mapping = {
    # Abbreviations
    "pb": "Punjab",
    "rj": "Rajasthan",
    "ar": "Arunachal Pradesh",
    "up": "Uttar Pradesh",
    "tn": "Tamil Nadu",
    "nl": "Nagaland",
    "mh": "Maharashtra",

    # Misspellings / variants
    "rajsthan": "Rajasthan",
    "rajshthan": "Rajasthan",
    "orissa": "Odisha",
    "pondicherry": "Puducherry",
    "nct of delhi": "Delhi",

    # Combined locations
    "punjab/mohali/zirakpur": "punjab",

    # Symbol variants
    "andaman & nicobar": "Andaman And Nicobar",
    "andaman and nicobar islands": "Andaman And Nicobar",
    "dadra & nagar haveli": "Dadra And Nagar Haveli",
    "dadra and nagar haveli": "Dadra And Nagar Haveli",
    "daman & diu": "Daman And Diu",
    "jammu & kashmir": "Jammu And Kashmir",
    "jammu and kashmir": "Jammu And Kashmir",
    "odisha": "Odisha",
    "delhi": "Delhi",
    "rajasthan": "Rajasthan"
}

df_cleaned["ship-state"] = df_cleaned["ship-state"].replace(state_mapping)

def camel_case(text):
    words = text.replace("&", "and").replace("-", " ").split()
    return " ".join(word.capitalize() for word in words)

df_cleaned["ship-state"] = df_cleaned["ship-state"].apply(camel_case)

In [ ]:
df_cleaned["ship-state"].unique()

## Calculating Sales Amount

In [ ]:
# Ensure Qty and Amount are numeric

df_cleaned["Qty"] = pd.to_numeric(df_cleaned["Qty"], errors="coerce")
df_cleaned["Amount"] = pd.to_numeric(df_cleaned["Amount"], errors="coerce")

# Create new column: Sales Amount
df_cleaned["Sales Amount"] = df_cleaned["Qty"] * df_cleaned["Amount"]
df_cleaned.head()

In [ ]:
numerical_cols = df_cleaned.select_dtypes(include='number')
numerical_cols

#### Train Test Split

In [ ]:
features = df_cleaned.drop(columns = ["Order ID", "Date", "SKU", "ASIN", "Sales Amount", "Size", "ship-city" ])
target = df_cleaned['Sales Amount']
print(target.head())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor(n_neighbors=10) # K = 10

In [ ]:
X_train.info()

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
print(f"The R2 of the model is {knn.score(X_test, y_test): .2f}")

In [ ]:
knn.predict(X_test)

In [ ]:
y_test.values